In [ ]:
import pandas as pd

import configparser
import os
from sqlalchemy import create_engine

import requests
from lxml import html

from datetime import datetime
import re
from bs4 import BeautifulSoup

config = configparser.ConfigParser()
config.read(os.path.join(os.path.expanduser('~'), '.config', 'me.ini'))

engine = create_engine(config['db']['url'])
pd.options.display.max_rows = 1000

import json

In [ ]:
headers = {
    'User-Agent': 'Mozilla/5.0',
    'Host': 'www.acesso.gov.pt'

}

session = requests.Session()

# Start page
r = session.get('https://www.acesso.gov.pt/jsp/loginRedirectForm.jsp?path=painelAdquirente.action&partID=EFPF', headers=headers)
cookies = requests.utils.cookiejar_from_dict(requests.utils.dict_from_cookiejar(session.cookies))

# Login
payload = {
    'username': config['efatura']['username'],
    'password': config['efatura']['password'],
    'partID': 'EFPF'
}
r = session.post('https://www.acesso.gov.pt/jsp/submissaoFormularioLogin', data=payload, cookies=cookies, headers=headers)

# get form for final redirect
payload = {}
for inp in BeautifulSoup(r.text, "lxml").find_all('form')[0].find_all('input'):
    if 'name' in inp.attrs:
        payload[inp['name']] = inp['value']

# Final redirect
headers = {
    'User-Agent': 'Mozilla/5.0',
    'Host': 'faturas.portaldasfinancas.gov.pt',
    'Origin': 'https://www.acesso.gov.pt',
    'Referer': 'https://www.acesso.gov.pt/jsp/submissaoFormularioLogin'

}
cookies = requests.utils.cookiejar_from_dict(requests.utils.dict_from_cookiejar(session.cookies))
r = session.post('https://faturas.portaldasfinancas.gov.pt/painelAdquirente.action', data=payload, cookies=cookies, headers=headers)

# Get the data
headers = {
    'User-Agent': 'Mozilla/5.0',
    'Host': 'faturas.portaldasfinancas.gov.pt'
}

cookies = requests.utils.cookiejar_from_dict(requests.utils.dict_from_cookiejar(session.cookies))
json_url = 'https://faturas.portaldasfinancas.gov.pt/json/obterDocumentosAdquirente.action?dataInicioFilter=2013-01-01&dataFimFilter=2016-01-01&ambitoAquisicaoFilter=TODOS'
values = session.get(json_url, cookies=cookies, headers=headers)

In [ ]:
intval = []
for dt in pd.date_range('2013-01-01', periods=4, freq='AS'):
    end_date = datetime(dt.year + 1, dt.month, dt.day)
    print('{} to {}'.format(dt.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d')))
    
    json_url = 'https://faturas.portaldasfinancas.gov.pt/json/obterDocumentosAdquirente.action?dataInicioFilter={:%Y-%m-%d}&dataFimFilter={:%Y-%m-%d}&ambitoAquisicaoFilter=TODOS'
    jsval = session.get(json_url.format(dt, end_date), cookies=cookies, headers=headers).json()
    intval.append(pd.DataFrame(jsval['linhas']))

In [ ]:
values = pd.concat(intval)

In [ ]:
len(values)

In [ ]:
values.head()

In [ ]:
values.columns

In [ ]:
values.to_sql('efatura', engine, if_exists='replace', index=False)

In [ ]:
vals = values[['dataEmissaoDocumento', 'nomeEmitente', 'valorTotal']]

In [ ]:
vals['valor'] = vals.valorTotal / 100

In [ ]:
vals.groupby('nomeEmitente').valor.sum().sort_values()